In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col
from pyspark.sql.functions import from_unixtime, hour, dayofyear
from pyspark.sql.functions import isnull, when, count, col
from pyspark.sql.functions import rank, col
from pyspark.sql.functions import *
from pyspark.sql import Window
import pyspark.sql.functions as f
import pyspark.sql.functions as F
import pandas as pd
from sparkmeasure import StageMetrics
import numpy as np
import random
from pyspark.sql.functions import broadcast
import re



spark = SparkSession \
.builder \
.appName('Vaseis2')\
.config("spark.jars", "<path-to-jar>/spark-measure_2.12-0.17.jar") \
.getOrCreate()


stagemetrics = StageMetrics(spark)


movies = (spark.read
      .format("csv")
      .option('header', 'true') #means that the first line contains column names
      .option("delimiter", ",") #set the delimiter to comma
      .option("inferSchema", "true") #automatically try to infer the column data types
      .load("movie.csv") #filename to read from
     )

ratings = (spark.read
      .format("csv")
      .option('header', 'true') #means that the first line contains column names
      .option("delimiter", ",") #set the delimiter to comma
      .option("inferSchema", "true") #automatically try to infer the column data types
      .load("rating.csv") #filename to read from
     )


tag = (spark.read
      .format("csv")
      .option('header', 'true') #means that the first line contains column names
      .option("delimiter", ",") #set the delimiter to comma
      .option("inferSchema", "true") #automatically try to infer the column data types
      .load("tag.csv") #filename to read from
     )


In [14]:
#1
stagemetrics.begin()
    
rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)
movie_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("movie.csv") #file to be processed
)
movie = movie_df.filter(movie_df["title"] == "Jumanji (1995)") #get rows with title jumanji
movieID = movie.collect()[0][0] #get movieID

q1 = rating_df.filter(rating_df["movieId"] == movieID).select(rating_df["userId"]).count() #count how many people have seen the movie jumanji 
#output
# 22243

stagemetrics.end()
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 7
numTasks => 29
elapsedTime => 4354 (4 s)
stageDuration => 4261 (4 s)
executorRunTime => 46764 (47 s)
executorCpuTime => 41818 (42 s)
executorDeserializeTime => 145 (0.1 s)
executorDeserializeCpuTime => 43 (43 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 1361 (1 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 2 (2 ms)
resultSize => 24252 (23.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 20049789
bytesRead => 1386473636 (1322.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 12
shuffleTotalBlocksFetched => 12
shuffleLocalBlocksFetched => 12
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 708 (708 Bytes)
shuffleLocalBytesRead => 708 (708 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 708 (708

In [15]:
#2
stagemetrics.begin()

tag_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("tag.csv") #file to be processed
)
movie_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("movie.csv") #file to be processed
)

boring_movies = tag_df.select('userId','movieId', lower(col('tag')).alias('tag')).filter(tag_df["tag"].contains('boring')).drop_duplicates(subset=['movieId']).select("movieId")
boring_movie_names = boring_movies.join(movie_df, boring_movies.movieId == movie_df.movieId).orderBy(movie_df['title']).select(movie_df['title'])




stagemetrics.end()

stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 4
numTasks => 9
elapsedTime => 226 (0.2 s)
stageDuration => 173 (0.2 s)
executorRunTime => 477 (0.5 s)
executorCpuTime => 453 (0.5 s)
executorDeserializeTime => 28 (28 ms)
executorDeserializeCpuTime => 17 (17 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 6 (6 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 9525 (9.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 492846
bytesRead => 23677914 (22.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRecordsWritt

In [3]:
#3
stagemetrics.begin()

tag_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("tag.csv") #file to be processed
)
rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)

bollywood_tag = tag_df.select('userId','movieId', lower(col('tag')).alias('tag')).filter(tag_df["tag"] == ('bollywood')).select('userId','movieId')
user_bollywood_good_rating = bollywood_tag.join(rating_df.select('userId','rating'), ["userId"], 'inner').distinct()
user_bollywood_good_rating = user_bollywood_good_rating.filter(user_bollywood_good_rating["rating"]>3.0).select('userId')

user_bollywood_good_rating.show()


stagemetrics.end()

stagemetrics.print_report()

+------+
|userId|
+------+
|124139|
|136015|
| 77137|
|113181|
|124139|
| 14517|
|  7671|
|124139|
| 14517|
| 20388|
| 93037|
| 77137|
| 79713|
|117681|
|124139|
| 14517|
| 62314|
| 93037|
|117681|
|    65|
+------+
only showing top 20 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 9
numTasks => 56
elapsedTime => 5397 (5 s)
stageDuration => 5017 (5 s)
executorRunTime => 53683 (54 s)
executorCpuTime => 48210 (48 s)
executorDeserializeTime => 295 (0.3 s)
executorDeserializeCpuTime => 136 (0.1 s)
resultSerializationTime => 1 (1 ms)
jvmGCTime => 2155 (2 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 70 (70 ms)
resultSize => 57492 (56.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 28336128
recordsRead => 32661510
bytesRead => 1427592728 (1361.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 39
shuffleTotalBlocksFetched => 32
shuffle

In [5]:
#4
from pyspark.sql.functions import year, row_number, col
from pyspark.sql.window import Window

stagemetrics.begin()

rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)
movie_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("movie.csv") #file to be processed
)

q4_df = rating_df.withColumn('year', year(rating_df['timestamp'])).groupBy('year','movieId').avg('rating')
window = Window.partitionBy(q4_df['year']).orderBy(q4_df['avg(rating)'].desc())

q4_df = q4_df.select('*', row_number().over(window).alias('row_num')).filter(col('row_num') <= 10).join(movie_df.select('movieId','title'),['movieId']).orderBy(movie_df['title'].asc()).select(movie_df['title'])

q4_df.show()

stagemetrics.end()

stagemetrics.print_report()

+--------------------+
|               title|
+--------------------+
|All Passion Spent...|
|And Nobody Weeps ...|
|Angel and the Bad...|
|Another Man's Poi...|
|       Aprile (1998)|
|     Arranged (2007)|
|Asoka (Ashoka the...|
|Attack of the Gia...|
|    Baby, The (1973)|
|Barren Lives (Vid...|
|Before the Fall (...|
|Better Living Thr...|
|Better Than Choco...|
| Bikini Beach (1964)|
|Bloody Territorie...|
|     Bob Funk (2009)|
|Bridge of Dragons...|
|Brief Vacation, A...|
|Brother of Sleep ...|
|Cabaret Balkan (B...|
+--------------------+
only showing top 20 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 8
numTasks => 428
elapsedTime => 7370 (7 s)
stageDuration => 7120 (7 s)
executorRunTime => 79406 (1.3 min)
executorCpuTime => 67112 (1.1 min)
executorDeserializeTime => 805 (0.8 s)
executorDeserializeCpuTime => 585 (0.6 s)
resultSerializationTime => 4 (4 ms)
jvmGCTime => 5140 (5 s)
shuffleFetchWaitTime 

In [8]:
#5
from pyspark.sql.functions import year, concat_ws, collect_list

stagemetrics.begin()

movie_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("movie.csv") #file to be processed
)
tag_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("tag.csv") #file to be processed
)

tag_df_2015 = tag_df.filter(year(tag_df['timestamp']) == 2015).select('movieId', 'tag').join(movie_df.select('movieId', 'title'), ['movieId'])
tag_df_2015 = tag_df_2015.groupBy('movieId','title').agg(concat_ws(", ", collect_list(tag_df_2015.tag)).alias('tags')).orderBy(tag_df_2015['title'].asc())

tag_df_2015.show()

stagemetrics.end()
stagemetrics.print_report()

+-------+--------------------+--------------------+
|movieId|               title|                tags|
+-------+--------------------+--------------------+
|  51372|""Great Performan...|                BD-R|
|   2072|  'burbs, The (1989)|1980's, black com...|
|  69757|(500) Days of Sum...|annoying, artisti...|
|  26216|...tick... tick.....|                BD-R|
| 128878|            1 (2014)|             Sukumar|
|   2572|10 Things I Hate ...|chick flick, Heat...|
|  58293|    10,000 BC (2008)|historically inac...|
|   6344|101 Reykjavik (10...|             Iceland|
|  27251|10th Kingdom, The...|SERIE DE TV, fant...|
|  90543|      11 x 14 (1977)|       James Benning|
|  90939|11-11-11 (11-11-1...|PG-13:some distur...|
|  27674|        11:14 (2003)|multiple storylin...|
|   1203| 12 Angry Men (1957)|group psychology,...|
|  77846| 12 Angry Men (1997)|             Bob*ola|
| 105844|12 Years a Slave ...|based on a book, ...|
|  65899|        12:01 (1993)|           time loop|
| 128844|   

In [9]:
#6
from pyspark.sql.functions import count

stagemetrics.begin()
movie_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("movie.csv") #file to be processed
)
rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)

q6_df = rating_df.groupBy('movieId').count()
q6_df = q6_df.join(movie_df.select('movieId','title'),['movieId']).orderBy(q6_df['count'].desc()).select(movie_df['title'])

stagemetrics.end()
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 4
numTasks => 15
elapsedTime => 2773 (3 s)
stageDuration => 2726 (3 s)
executorRunTime => 31291 (31 s)
executorCpuTime => 27279 (27 s)
executorDeserializeTime => 1 (1 ms)
executorDeserializeCpuTime => 9 (9 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 1625 (2 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 19435 (18.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 20027545
bytesRead => 693302354 (661.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRecords

In [10]:
#7
from pyspark.sql.window import Window
from pyspark.sql.functions import year, row_number, col, count

stagemetrics.begin()
rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)

q7_df = rating_df.withColumn('year', year(rating_df['timestamp'])).groupBy('year','userId').count()
window = Window.partitionBy(q7_df['year']).orderBy(q7_df['count'].desc())
q7_df = q7_df.select('*', row_number().over(window).alias('row_num')).filter(col('row_num') <= 10).select('userId')

q7_df.show()

stagemetrics.end()
stagemetrics.print_report()

+------+
|userId|
+------+
| 72983|
| 33736|
| 21861|
| 13623|
|  2261|
| 67346|
|113991|
|134401|
| 50943|
| 45637|
|  8405|
| 71975|
|118754|
| 31404|
| 69793|
|   741|
|120575|
| 61168|
| 34587|
| 91893|
+------+
only showing top 20 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 7
numTasks => 250
elapsedTime => 5897 (6 s)
stageDuration => 5786 (6 s)
executorRunTime => 66238 (1.1 min)
executorCpuTime => 59529 (60 s)
executorDeserializeTime => 137 (0.1 s)
executorDeserializeCpuTime => 204 (0.2 s)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 3072 (3 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 1851 (2 s)
resultSize => 797124 (778.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 262144000
recordsRead => 40000528
bytesRead => 1383420804 (1319.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 223818
shuffleTotalBlocksFetched =>

In [12]:
#8
from pyspark.sql.window import Window
from pyspark.sql.functions import year, row_number, col, count, split, explode

stagemetrics.begin()
movie_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("movie.csv") #file to be processed
)
rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)

q8_df = movie_df.withColumn('genres',explode(split('genres',"[^\w]"))) #seperates rows for each genre in a movie
q8_df = q8_df.join(rating_df,['movieId'],'left')
q8_df = q8_df.groupBy('genres','title').count()
window = Window.partitionBy(q8_df['genres']).orderBy(q8_df['count'].desc())
q8_df = q8_df.select('*', row_number().over(window).alias('r')).filter(col('r') <= 5)

q8_df.show()

stagemetrics.end()
stagemetrics.print_report()

+--------+--------------------+-----+---+
|  genres|               title|count|  r|
+--------+--------------------+-----+---+
|   Crime| Pulp Fiction (1994)|67310|  1|
|   Crime|Shawshank Redempt...|63366|  2|
|   Crime|Silence of the La...|63299|  3|
|   Crime|Usual Suspects, T...|47006|  4|
|   Crime|       Batman (1989)|46054|  5|
| Romance| Forrest Gump (1994)|66172|  1|
| Romance|    True Lies (1994)|43159|  2|
| Romance|        Speed (1994)|41562|  3|
| Romance|Beauty and the Be...|35138|  4|
| Romance| Pretty Woman (1990)|33900|  5|
|Thriller| Pulp Fiction (1994)|67310|  1|
|Thriller|Silence of the La...|63299|  2|
|Thriller|Jurassic Park (1993)|59715|  3|
|Thriller|  Matrix, The (1999)|51334|  4|
|Thriller|Fugitive, The (1993)|49581|  5|
|      Fi|Jurassic Park (1993)|59715|  1|
|      Fi|Star Wars: Episod...|54502|  2|
|      Fi|Terminator 2: Jud...|52244|  3|
|      Fi|  Matrix, The (1999)|51334|  4|
|      Fi|Independence Day ...|47048|  5|
+--------+--------------------+---

In [22]:
9.
from pyspark.sql.window import Window
from pyspark.sql.functions import year, row_number, col, count, split, explode, rank, dayofyear, hour

stagemetrics.begin()
rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)

q9_df = rating_df.withColumn('year', year(rating_df['timestamp']))
q9_df = q9_df.withColumn('doy', dayofyear(rating_df['timestamp']))
q9_df = q9_df.withColumn('hour', hour(rating_df['timestamp']))
q9_df = q9_df.groupBy(q9_df['movieId'],q9_df['year'],q9_df['doy'],q9_df['hour']).count()
q9 = q9_df.filter(q9_df['count']>1).select(sum('count'))
stagemetrics.end()
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 2
numTasks => 13
elapsedTime => 2727 (3 s)
stageDuration => 2716 (3 s)
executorRunTime => 30408 (30 s)
executorCpuTime => 26671 (27 s)
executorDeserializeTime => 62 (62 ms)
executorDeserializeCpuTime => 22 (22 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 1335 (1 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 19908 (19.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 20000265
bytesRead => 691743170 (659.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRec

In [23]:
#10
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, count

stagemetrics.begin()
rating_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("rating.csv") #file to be processed
)
movie_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("movie.csv") #file to be processed
)

tag_df = (spark.read
    .format("csv")
    .option('header','true') #csv files have headers
    .option('delimiter', ',') #delimiter of the csv files that are processed
    .option('inferSchema', 'true') #if false, all values are strings
    .load("tag.csv") #file to be processed
)
q10_df = movie_df.withColumn('genres',explode(split('genres',"[^\w]"))) #seperates rows for each genre in a movie
q10_df = q10_df.join(tag_df,['movieId'],'left')
q10_df = q10_df.select('*',lower(col('tag')).alias('tag')).filter(q10_df["tag"] == "funny")
q10_df = q10_df.join(rating_df,['movieId'],'left')
q10_df = q10_df.filter(q10_df['rating'] >= 3.5).select("*")
q10_df = q10_df.dropDuplicates(['movieId'])
q10_df = q10_df.groupBy(q10_df['genres']).count().orderBy(q10_df['genres'].asc())

stagemetrics.end()
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 12
Aggregated Spark stage metrics:
numStages => 6
numTasks => 22
elapsedTime => 2941 (3 s)
stageDuration => 2851 (3 s)
executorRunTime => 30012 (30 s)
executorCpuTime => 27506 (28 s)
executorDeserializeTime => 74 (74 ms)
executorDeserializeCpuTime => 34 (34 ms)
resultSerializationTime => 0 (0 ms)
jvmGCTime => 1099 (1 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 0 (0 ms)
resultSize => 19908 (19.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 0
recordsRead => 20493111
bytesRead => 715421084 (682.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 0
shuffleTotalBlocksFetched => 0
shuffleLocalBlocksFetched => 0
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 0 (0 Bytes)
shuffleLocalBytesRead => 0 (0 Bytes)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 0 (0 Bytes)
shuffleRec

In [12]:
spark